# DataFrames: Read and Write Data
     
Dask Dataframes can read and store data in many of the same formats as Pandas dataframes.  In this example we read and write data with the popular CSV and Parquet formats, and discuss best practices when using these formats.

In [1]:
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/0eEsIA0O1iE?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

/home/travis/miniconda/envs/test/lib/python3.8/site-packages/IPython/core/display.py:717: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


## Start Dask Client for Dashboard

Starting the Dask Client is optional.  It will provide a dashboard which 
is useful to gain insight on the computation.  

The link to the dashboard will become visible when you create the client below.  We recommend having it open on one side of your screen while using your notebook on the other side.  This can take some effort to arrange your windows, but seeing them both at the same is very useful when learning.

In [2]:
from dask.distributed import Client
client = Client(n_workers=1, threads_per_worker=4, processes=False, memory_limit='2GB')
client

<Client: 'inproc://10.20.0.76/10559/1' processes=1 threads=4, memory=2.00 GB>

## Create artificial dataset

First we create an artificial dataset and write it to many CSV files.

You don't need to understand this section, we're just creating a dataset for the rest of the notebook.

In [3]:
import dask
df = dask.datasets.timeseries()
df

Dask DataFrame Structure:
                   id    name        x        y
npartitions=30                                 
2000-01-01      int64  object  float64  float64
2000-01-02        ...     ...      ...      ...
...               ...     ...      ...      ...
2000-01-30        ...     ...      ...      ...
2000-01-31        ...     ...      ...      ...
Dask Name: make-timeseries, 30 tasks

In [4]:
import os
import datetime

if not os.path.exists('data'):
    os.mkdir('data')

def name(i):
    """ Provide date for filename given index
    
    Examples
    --------
    >>> name(0)
    '2000-01-01'
    >>> name(10)
    '2000-01-11'
    """
    return str(datetime.date(2000, 1, 1) + i * datetime.timedelta(days=1))
    
df.to_csv('data/*.csv', name_function=name);

## Read CSV files

We now have many CSV files in our data directory, one for each day in the month of January 2000.  Each CSV file holds timeseries data for that day.  We can read all of them as one logical dataframe using the `dd.read_csv` function with a glob string.

In [5]:
!ls data/*.csv | head

data/2000-01-01.csv
data/2000-01-02.csv
data/2000-01-03.csv
data/2000-01-04.csv
data/2000-01-05.csv
data/2000-01-06.csv
data/2000-01-07.csv
data/2000-01-08.csv
data/2000-01-09.csv
data/2000-01-10.csv


In [6]:
!head data/2000-01-01.csv

timestamp,id,name,x,y
2000-01-01 00:00:00,1036,Xavier,-0.5994994109985681,-0.9056550553165863
2000-01-01 00:00:01,1024,Tim,-0.8924991967767719,-0.9328972162315017
2000-01-01 00:00:02,1019,George,-0.6388234895019453,0.079455516571451
2000-01-01 00:00:03,981,Xavier,0.5248693832613154,-0.44171238484841924
2000-01-01 00:00:04,1001,Victor,0.33790012406837366,-0.0923019026599119
2000-01-01 00:00:05,996,Dan,0.6705987903671147,-0.02307272691654383
2000-01-01 00:00:06,966,Norbert,0.634346525245151,-0.5689276804571424
2000-01-01 00:00:07,1007,Yvonne,-0.05425749783915523,0.42345206808411495
2000-01-01 00:00:08,1014,Yvonne,0.7535767023849018,0.35771678289010533


In [7]:
!head data/2000-01-30.csv

timestamp,id,name,x,y
2000-01-30 00:00:00,973,Xavier,-0.029200023495674188,-0.6237961595490611
2000-01-30 00:00:01,1024,Norbert,-0.38890327797245083,0.4871633870029952
2000-01-30 00:00:02,993,Frank,0.7401367359114586,-0.5600311774182611
2000-01-30 00:00:03,1012,Jerry,-0.6374609239313087,-0.5830007009099989
2000-01-30 00:00:04,984,Kevin,0.7443865018551905,-0.40750502888546736
2000-01-30 00:00:05,1037,Wendy,-0.5196237602057785,-0.3648323837768932
2000-01-30 00:00:06,955,George,0.8708395387164689,-0.9602260356362804
2000-01-30 00:00:07,970,Norbert,-0.37734615370785773,-0.5468558957341743
2000-01-30 00:00:08,1032,Sarah,0.6376133491116189,-0.9175414440186698


We can read one file with `pandas.read_csv` or many files with `dask.dataframe.read_csv`

In [8]:
import pandas as pd

df = pd.read_csv('data/2000-01-01.csv')
df.head()

timestamp    id    name         x         y
0  2000-01-01 00:00:00  1036  Xavier -0.599499 -0.905655
1  2000-01-01 00:00:01  1024     Tim -0.892499 -0.932897
2  2000-01-01 00:00:02  1019  George -0.638823  0.079456
3  2000-01-01 00:00:03   981  Xavier  0.524869 -0.441712
4  2000-01-01 00:00:04  1001  Victor  0.337900 -0.092302

In [9]:
import dask.dataframe as dd

df = dd.read_csv('data/2000-*-*.csv')
df

Dask DataFrame Structure:
               timestamp     id    name        x        y
npartitions=30                                           
                  object  int64  object  float64  float64
                     ...    ...     ...      ...      ...
...                  ...    ...     ...      ...      ...
                     ...    ...     ...      ...      ...
                     ...    ...     ...      ...      ...
Dask Name: read-csv, 30 tasks

In [10]:
df.head()

timestamp    id    name         x         y
0  2000-01-01 00:00:00  1036  Xavier -0.599499 -0.905655
1  2000-01-01 00:00:01  1024     Tim -0.892499 -0.932897
2  2000-01-01 00:00:02  1019  George -0.638823  0.079456
3  2000-01-01 00:00:03   981  Xavier  0.524869 -0.441712
4  2000-01-01 00:00:04  1001  Victor  0.337900 -0.092302

## Tuning read_csv

The Pandas `read_csv` function has *many* options to help you parse files.  The Dask version uses the Pandas function internally, and so supports many of the same options.  You can use the `?` operator to see the full documentation string.

In [11]:
pd.read_csv?

In [12]:
dd.read_csv?

In this case we use the `parse_dates` keyword to parse the timestamp column to be a datetime.  This will make things more efficient in the future.  Notice that the dtype of the timestamp column has changed from `object` to `datetime64[ns]`.

In [13]:
df = dd.read_csv('data/2000-*-*.csv', parse_dates=['timestamp'])
df

Dask DataFrame Structure:
                     timestamp     id    name        x        y
npartitions=30                                                 
                datetime64[ns]  int64  object  float64  float64
                           ...    ...     ...      ...      ...
...                        ...    ...     ...      ...      ...
                           ...    ...     ...      ...      ...
                           ...    ...     ...      ...      ...
Dask Name: read-csv, 30 tasks

## Do a simple computation

Whenever we operate on our dataframe we read through all of our CSV data so that we don't fill up RAM.  This is very efficient for memory use, but reading through all of the CSV files every time can be slow.

In [14]:
%time df.groupby('name').x.mean().compute()

CPU times: user 4.52 s, sys: 615 ms, total: 5.13 s
Wall time: 3.41 s


name
Alice       0.002621
Bob        -0.002344
Charlie     0.000022
Dan         0.000978
Edith      -0.003836
Frank      -0.002697
George     -0.001981
Hannah      0.002244
Ingrid     -0.000526
Jerry       0.001565
Kevin       0.001706
Laura       0.001938
Michael     0.000087
Norbert    -0.002065
Oliver     -0.001521
Patricia   -0.001788
Quinn       0.000534
Ray        -0.000668
Sarah       0.000899
Tim        -0.001340
Ursula     -0.002410
Victor     -0.000043
Wendy       0.003106
Xavier      0.003621
Yvonne     -0.001845
Zelda      -0.001481
Name: x, dtype: float64

## Write to Parquet

Instead, we'll store our data in Parquet, a format that is more efficient for computers to read and write.

In [15]:
df.to_parquet('data/2000-01.parquet', engine='pyarrow')

In [16]:
!ls data/2000-01.parquet/

_common_metadata  part.15.parquet  part.22.parquet  part.2.parquet
_metadata	  part.16.parquet  part.23.parquet  part.3.parquet
part.0.parquet	  part.17.parquet  part.24.parquet  part.4.parquet
part.10.parquet   part.18.parquet  part.25.parquet  part.5.parquet
part.11.parquet   part.19.parquet  part.26.parquet  part.6.parquet
part.12.parquet   part.1.parquet   part.27.parquet  part.7.parquet
part.13.parquet   part.20.parquet  part.28.parquet  part.8.parquet
part.14.parquet   part.21.parquet  part.29.parquet  part.9.parquet


## Read from Parquet

In [17]:
df = dd.read_parquet('data/2000-01.parquet', engine='pyarrow')
df

Dask DataFrame Structure:
                     timestamp     id    name        x        y
npartitions=30                                                 
                datetime64[ns]  int64  object  float64  float64
                           ...    ...     ...      ...      ...
...                        ...    ...     ...      ...      ...
                           ...    ...     ...      ...      ...
                           ...    ...     ...      ...      ...
Dask Name: read-parquet, 30 tasks

In [18]:
%time df.groupby('name').x.mean().compute()

CPU times: user 1.39 s, sys: 142 ms, total: 1.53 s
Wall time: 1.04 s


name
Alice       0.002621
Bob        -0.002344
Charlie     0.000022
Dan         0.000978
Edith      -0.003836
Frank      -0.002697
George     -0.001981
Hannah      0.002244
Ingrid     -0.000526
Jerry       0.001565
Kevin       0.001706
Laura       0.001938
Michael     0.000087
Norbert    -0.002065
Oliver     -0.001521
Patricia   -0.001788
Quinn       0.000534
Ray        -0.000668
Sarah       0.000899
Tim        -0.001340
Ursula     -0.002410
Victor     -0.000043
Wendy       0.003106
Xavier      0.003621
Yvonne     -0.001845
Zelda      -0.001481
Name: x, dtype: float64

## Select only the columns that you plan to use

Parquet is a column-store, which means that it can efficiently pull out only a few columns from your dataset.  This is good because it helps to avoid unnecessary data loading.

In [19]:
%%time
df = dd.read_parquet('data/2000-01.parquet', columns=['name', 'x'], engine='pyarrow')
df.groupby('name').x.mean().compute()

CPU times: user 1.31 s, sys: 87.3 ms, total: 1.4 s
Wall time: 1.02 s


name
Alice       0.002621
Bob        -0.002344
Charlie     0.000022
Dan         0.000978
Edith      -0.003836
Frank      -0.002697
George     -0.001981
Hannah      0.002244
Ingrid     -0.000526
Jerry       0.001565
Kevin       0.001706
Laura       0.001938
Michael     0.000087
Norbert    -0.002065
Oliver     -0.001521
Patricia   -0.001788
Quinn       0.000534
Ray        -0.000668
Sarah       0.000899
Tim        -0.001340
Ursula     -0.002410
Victor     -0.000043
Wendy       0.003106
Xavier      0.003621
Yvonne     -0.001845
Zelda      -0.001481
Name: x, dtype: float64

Here the difference is not that large, but with larger datasets this can save a great deal of time.

## Learn more

http://dask.pydata.org/en/latest/dataframe-create.html